In [1]:
from skmob.models.epr import DensityEPR, SpatialEPR
import skmob
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
# load a spatial tesellation on which to perform the simulation
url = skmob.utils.constants.NY_COUNTIES_2011
tessellation = gpd.read_file(url)
# starting and end times of the simulation
start_time = pd.to_datetime('2020/08/13 00:00:00')
true_start_time = pd.to_datetime('2020/08/20 00:00:00')
end_time = pd.to_datetime('2020/09/03 00:00:00')

## DensityEPR

In [5]:
## override if necessary
class ConstantTimeDensityEPR(DensityEPR):
    def __init__(self):
        super().__init__()
        
        
def apply_minute(tdf, minutes=10):
    unix_minutes = minutes * 60
    return tdf['time'].apply(lambda x: x - (x % unix_minutes))

def gauusian_bridge(tdf, uid, random_state, minutes):
    unix_minutes = minutes * 60
    tmp_dict = {"uid": [], "time": [], "lat": [], "lng": []}
    table_size = len(tdf)
    for i in range(-1, table_size):
        if (i == table_size - 1):
            time_diff = (int(end_time.timestamp()) - tdf['time'].iloc[i])
            curr_time = tdf['time'].iloc[i]
            i -= 10
        elif (i == -1):
            time_diff = tdf['time'].iloc[0] - int(true_start_time.timestamp())
            curr_time = int(true_start_time.timestamp())
            i += 10
        else:
            time_diff = (tdf['time'].iloc[i+1] - tdf['time'].iloc[i])
            curr_time = tdf['time'].iloc[i]
        
        time_times = time_diff // unix_minutes
        lat_diff = (tdf['lat'].iloc[i+1] - tdf['lat'].iloc[i]) / time_times
        lng_diff = (tdf['lng'].iloc[i+1] - tdf['lng'].iloc[i]) / time_times
        
        curr_lat = tdf['lat'].iloc[i]
        curr_lng = tdf['lng'].iloc[i]
        
        random_walk = random_state.random(time_times)
        lat_noise = random_state.normal(lat_diff, np.abs(lat_diff/10), time_times)
        curr_lat += lat_noise[time_times-1]
        lng_noise = random_state.normal(lng_diff, np.abs(lng_diff/10), time_times)
        curr_lng += lng_noise[time_times-1]
        
        for j in range(time_times):
            tmp_dict["uid"].append(uid)
            tmp_dict["time"].append(curr_time)
            tmp_dict["lat"].append(curr_lat)
            tmp_dict["lng"].append(curr_lng)
            
            curr_time += unix_minutes
            if random_walk[j] < 0.5:
                curr_lat += lat_noise[j]
                curr_lng += lng_noise[j]

    df = pd.DataFrame.from_dict(tmp_dict)
    return df

def for_all_user(tdf, random_state, minutes=10):
    df_list = []
    for uid in tdf.groupby('uid').groups.keys():
        df_list.append(gauusian_bridge(tdf[tdf['uid'] == uid], uid, random_state, minutes))
    return pd.concat(df_list, axis=0)

## Server data

In [ ]:
agents = 100
seed = 1
minutes = 10

state = np.random.RandomState(seed)
start_locations = list(state.choice(list(range(0,62)), agents, True))
depr = ConstantTimeDensityEPR()

# start the simulation
tdf = depr.generate(start_time, end_time, tessellation, relevance_column='population', n_agents=agents, random_state=seed, show_progress=True, starting_locations=start_locations)

tdf = tdf[tdf['datetime'] >= true_start_time]
tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
tdf['time'] = apply_minute(tdf, minutes=minutes)
tdf = for_all_user(tdf, state, minutes)
tdf[['time', 'lat', 'lng']].to_csv(f'NY-DensityEPR-{minutes}-{seed}-{agents}.csv', index=False)
tdf

## Client data

In [7]:
client_size = 1000
agents = 1
seed = 1
minutes = 10

state = np.random.RandomState(seed)

for i in tqdm(range(client_size)):
    start_locations = list(state.choice(list(range(0,62)), agents, True))
    depr = ConstantTimeDensityEPR()
    # start the simulation
    tdf = depr.generate(start_time, end_time, tessellation, relevance_column='population', n_agents=agents, random_state=seed, show_progress=True, starting_locations=start_locations)

    tdf = tdf[tdf['datetime'] >= true_start_time]
    tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
    tdf['time'] = apply_minute(tdf, minutes=minutes)
    tdf = for_all_user(tdf, state, minutes)
    tdf[['time', 'lat', 'lng']].to_csv(f'client/client-{i}-NY-DensityEPR-{minutes}-{seed}-{agents}.csv', index=False)
tdf

<ipython-input-7-1729c62a331b>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(client_size)):


  0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]/Users/fumiyuki/.pyenv/versions/3.9.1/envs/trajectory/lib/python3.9/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████| 1/1 [00:00<00:00, 11.65it/s]
<ipython-input-7-1729c62a331b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
<ipython-input-7-1729c62a331b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['ti


  0%|          | 0/1 [00:00<?, ?it/s]/Users/fumiyuki/.pyenv/versions/3.9.1/envs/trajectory/lib/python3.9/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████| 1/1 [00:00<00:00, 11.88it/s]
<ipython-input-7-1729c62a331b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
<ipython-input-7-1729c62a331b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['ti


  0%|          | 0/1 [00:00<?, ?it/s]/Users/fumiyuki/.pyenv/versions/3.9.1/envs/trajectory/lib/python3.9/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████| 1/1 [00:00<00:00, 11.94it/s]
<ipython-input-7-1729c62a331b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
<ipython-input-7-1729c62a331b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['ti

...

  0%|          | 0/1 [00:00<?, ?it/s]/Users/fumiyuki/.pyenv/versions/3.9.1/envs/trajectory/lib/python3.9/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)

100%|██████████| 1/1 [00:00<00:00,  4.71it/s]
<ipython-input-7-1729c62a331b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
<ipython-input-7-1729c62a331b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['ti

  0%|          | 0/1 [00:00<?, ?it/s]/Users/fumiyuki/.pyenv/versions/3.9.1/envs/trajectory/lib/python3.9/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)

100%|██████████| 1/1 [00:00<00:00,  4.68it/s]
<ipython-input-7-1729c62a331b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
<ipython-input-7-1729c62a331b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['ti

  0%|          | 0/1 [00:00<?, ?it/s]/Users/fumiyuki/.pyenv/versions/3.9.1/envs/trajectory/lib/python3.9/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)

100%|██████████| 1/1 [00:00<00:00,  4.73it/s]
<ipython-input-7-1729c62a331b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
<ipython-input-7-1729c62a331b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['ti


  0%|          | 0/1 [00:00<?, ?it/s]/Users/fumiyuki/.pyenv/versions/3.9.1/envs/trajectory/lib/python3.9/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)

100%|██████████| 1/1 [00:00<00:00,  4.92it/s]
<ipython-input-7-1729c62a331b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['time'] = tdf['datetime'].apply(lambda x: int(x.timestamp()))
<ipython-input-7-1729c62a331b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['t

,uid,time,lat,lng
0,1,1597881600,40.849266,-73.966397
1,1,1597882200,40.849266,-73.966397
2,1,1597882800,40.919096,-73.963374
3,1,1597883400,40.919096,-73.963374
4,1,1597884000,40.986090,-73.960513
...,...,...,...,...
2011,1,1599088200,40.939218,-73.788265
2012,1,1599088800,40.939218,-73.788265
2013,1,1599089400,40.954586,-73.785798
2014,1,1599090000,40.967042,-73.783246
